# This notebook is mainly used for Coursera Appliced Data Science Capstone

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


##Crawling data from wikipedia using beautiful soup package

In [11]:
import requests
from bs4 import BeautifulSoup
# import pandas as pd
# import numpy as np

# get the page
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# parse the page
html=response.text
soup=BeautifulSoup(html, "html.parser")

# use the first table
tables = soup.find_all("table")
postal_code_table = tables[0]
table_headers = []
for col in postal_code_table.find_all("th"):
  # print("  header: '%s'" % col.text.strip())
  table_headers.append(col.text.strip())
print("headers:", table_headers)

# getting rows, and cells per row
rawdata = []
rows = postal_code_table.find_all("tr")
count = 0
for r in rows:
  count += 1
  cells = r.find_all("td")

  # ignore header / invalid rows
  if (len(cells) == 0): continue

  postalcode    = cells[0].text.strip()
  borough       = cells[1].text.strip()
  neighborhood  = cells[2].text.strip()

  # ignore Borough "Not assigned"
  if (borough == "Not assigned"): continue

  # "Not assigned" neighborhood makes neighborhood is same as borough
  if (neighborhood == "Not assigned"):
    print("%s --> %s" % (neighborhood, borough))
    neighborhood = borough

  rawdata.append([postalcode, borough, neighborhood])

print("total records:", len(rawdata))
df = pd.DataFrame(rawdata, columns = table_headers) 
# print(len(df[table_headers[0]].values))
# print(df)
print(df.head(10))

print("dataset shape: ", df.shape)

headers: ['Postal Code', 'Borough', 'Neighbourhood']
total records: 103
  Postal Code           Borough                                Neighbourhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
5         M9A         Etobicoke      Islington Avenue, Humber Valley Village
6         M1B       Scarborough                               Malvern, Rouge
7         M3B        North York                                    Don Mills
8         M4B         East York              Parkview Hill, Woodbine Gardens
9         M5B  Downtown Toronto                     Garden District, Ryerson
dataset shape:  (103, 3)


## Getting geo lat long

In [17]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

--2020-08-16 09:40:32--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.55.161.7
Connecting to cocl.us (cocl.us)|169.55.161.7|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-16 09:40:34--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 103.116.4.197
Connecting to ibm.box.com (ibm.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-16 09:40:34--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-16 09:40:

In [31]:
# read from the files
# !wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data
geolatlongdf = pd.read_csv("./Geospatial_Coordinates.csv")
print("lat long dataset:")
print(geolatlongdf.head())
print(geolatlongdf.shape)

# merge geolatlongdf and df together
dfmerged = df.merge(geolatlongdf, how="left", on=table_headers[0])
print("Merged dataset:")
dfmerged.head()
# print(dfmerged.shape)

lat long dataset:
  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
(103, 3)
Merged dataset:


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
